In [8]:
import cv2
from PIL import Image, ImageOps
import numpy as np

img1 = cv2.imread("../static_files/test_image.jpg")
print(img1.shape)

img2 = Image.open("../static_files/test_image.jpg")  
img2 = np.asarray(img2)
img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2BGR)
print(img2.shape)


(640, 640, 3)
(640, 640, 3)


In [21]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=False, force_reload=True, autoshape=True) 


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/caveman/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-12-15 Python-3.10.6 torch-1.11.0+cu102 CPU


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                       

In [20]:
torch.save(model, '.yolov5s_v2.pt') 

In [22]:
model = torch.load('.yolov5s_v2.pt')

In [26]:
model.classes = [3]

In [27]:
model(img1)

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 640x640 4 motorcycles
Speed: 6.5ms pre-process, 379.6ms inference, 3.3ms NMS per image at shape (1, 3, 640, 640)

In [6]:
from deepsparse.pipelines.custom_pipeline import CustomTaskPipeline
from deepsparse import Pipeline
import json,os

static_file_path= '../static_files/yolo_classes.json'
try:
    with open(static_file_path, 'r') as fp:
        key_to_string = json.load(fp)
    string_to_key = {value:key for key, value in key_to_string.items()}
except Exception as e:
    raise Exception(f"Error {e} while loading model class")

kwargs = {}
# ***************************** initialize YOLO-V5 **********************************
# self.detector = torch.load(kwargs.get('weights','yolov5/weights/yolov5s.pt'), map_location=self.device)['model'].float()  # load to FP32
model_size = kwargs.get('model_size','yolov5l-p') #['yolov5-s', 'yolov5-l']
model_weight_path = kwargs.get('model_weight_path', "./model_weight/"+model_size+"_v2.onnx") #'yolov5/weights/yolov5s.pt') "./model_weight/yolov5s.pt"
pretrained_model = False if os.path.exists(model_weight_path) else True
stub = {
            "yolov5-l":"zoo:cv/detection/yolov5-l/pytorch/ultralytics/coco/pruned_quant-aggressive_95",
            "yolov5-s":"zoo:cv/detection/yolov5-s/pytorch/ultralytics/coco/pruned_quant-aggressive_94",
            'yolov5l-p':"zoo:cv/detection/yolov5-l/pytorch/ultralytics/coco/pruned-aggressive_98",
            'yolov5l-pq':"zoo:cv/detection/yolov5-l/pytorch/ultralytics/coco/pruned_quant-aggressive_95",

        }
print(model_weight_path)
yolo_pipeline = Pipeline.create(
task="yolo",
model_path= stub.get(model_size, None) if pretrained_model else model_weight_path,
class_names=list(key_to_string.keys()),   # if using custom model, pass in a list of classes the model will clasify or a path to a json file containing them
model_config=None,  # if using custom model, pass in the path to a local model config file here
)

./model_weight/yolov5l-p_v2.onnx


DecodeError: Error parsing message with type 'onnx.ModelProto'

In [12]:
if not os.path.isdir("model_weight"):
    os.makedirs("model_weight")

if os.path.isfile(yolo_pipeline.onnx_file_path):
    os.rename(yolo_pipeline.onnx_file_path, model_weight_path) if pretrained_model else None

In [13]:
yolo_pipeline.onnx_file_path

'./model_weight/yolov5-s_v2.onnx'

In [43]:
yolo_pipeline.output_schema()

ValidationError: 3 validation errors for YOLOOutput
boxes
  field required (type=value_error.missing)
scores
  field required (type=value_error.missing)
labels
  field required (type=value_error.missing)

In [47]:
import cv2
import numpy as np
image = cv2.imread("../static_files/test_image.jpg")
image.astype(np.uint8)
print(image.shape)
# yolo_pipeline.class_names([2,3])
pipeline_outputs = yolo_pipeline(images=image, iou_thres=0.6, conf_thres=0.7)

(640, 640, 3)


In [22]:
pipeline_outputs

YOLOOutput(boxes=[[[45.29338073730469, 168.49630737304688, 228.07969665527344, 311.3185119628906], [76.69711303710938, 32.82611083984375, 199.724365234375, 131.873291015625], [63.94458770751953, 485.4278564453125, 213.818115234375, 621.5233154296875], [240.37136840820312, 541.4109497070312, 394.8977966308594, 614.7621459960938], [252.99996948242188, 40.96147155761719, 404.4091491699219, 129.52249145507812], [421.9398498535156, 201.61544799804688, 581.795166015625, 300.8406066894531], [430.152099609375, 519.798583984375, 578.988525390625, 620.963623046875], [75.14987182617188, 325.3128662109375, 195.07424926757812, 470.36883544921875], [460.8572082519531, 352.4356689453125, 578.5552978515625, 451.70166015625], [426.97796630859375, 15.077621459960938, 588.6062622070312, 147.41343688964844], [245.1121826171875, 197.54347229003906, 390.9814453125, 287.2884521484375], [242.71847534179688, 342.217529296875, 400.8564147949219, 463.16705322265625]]], scores=[[0.934207558631897, 0.9156649708747